In [1]:
import time

import numpy as np
from numba import njit
from SALib.analyze import sobol
from SALib.sample import saltelli

# Проведите анализ чувствительности, проверьте сходимость, измерьте тайминги

In [2]:
NUM_ITERATIONS = 2 ** 17
NUM_VARIABLES = 5

In [3]:
problem = {
    "num_vars": NUM_VARIABLES,
    "names": [f"x{i + 1}" for i in range(NUM_VARIABLES)],
    "bounds": [[-1.0, 1.0] for _ in range(NUM_VARIABLES)]
}

In [4]:
def evaluate_model(x):
    result = x[0] ** 5 \
             - x[0]**4 * x[1] \
             + x[0]**3 * x[1]**2 * x[2] \
             - x[0]**2 * x[1]**3 * x[2]**2 * x[3] \
             + x[0]    * x[1]**4 * x[2]**3 * x[3]**2 * x[4]
    return result

In [5]:
start_time = time.time()
param_values = saltelli.sample(problem, NUM_ITERATIONS)

print(f"Samples generation took {time.time() - start_time:.6f} seconds")
print(f"param_values shape: {param_values.shape}")

/var/folders/6s/19zl7nr54jn29b08ttwznqbm0000gp/T/ipykernel_4946/1610289144.py:2: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, NUM_ITERATIONS)


Samples generation took 7.454577 seconds
param_values shape: (1572864, 5)


In [6]:
y = np.zeros([param_values.shape[0]])
start_time = time.time()

for i, x in enumerate(param_values):
    y[i] = evaluate_model(x)

eval_time = time.time() - start_time
print(f"Model evaluation took {eval_time:.6f} seconds")

Model evaluation took 4.015293 seconds


In [7]:
start_time = time.time()
sobol_indices = sobol.analyze(problem, y)
print(f"Sobol analysis took {time.time() - start_time:.6f} seconds")

Sobol analysis took 25.532118 seconds


In [8]:
print("First-order indices:")
print(sobol_indices["S1"])

print("\nSecond-order indices:")
for i in range(NUM_VARIABLES):
    for j in range(i + 1, NUM_VARIABLES):
        print(f"x{i + 1}-x{j + 1}: {sobol_indices['S2'][i, j]}")

print("\nTotal indices for each variable:")
print(sobol_indices["ST"])

First-order indices:
[6.50612740e-01 9.58546455e-02 2.04748855e-04 2.46264223e-05
 1.06867457e-04]

Second-order indices:
x1-x2: 0.16987438126484783
x1-x3: 0.03826275403003994
x1-x4: 0.00020946561717304196
x1-x5: 0.0004444833633367217
x2-x3: -0.00011958479017451526
x2-x4: 0.004237998244049233
x2-x5: -7.914446571177359e-05
x3-x4: -4.597263128522027e-05
x3-x5: 0.00025829959606260956
x4-x5: 0.00041373804245725166

Total indices for each variable:
[0.89695428 0.31024338 0.07663299 0.01500546 0.00255642]


# Ускорьте вычисления Python с использованием numba

In [9]:
@njit
def evaluate_model(x):
    result = x[0]**5 \
             - x[0]**4 * x[1] \
             + x[0]**3 * x[1]**2 * x[2] \
             - x[0]**2 * x[1]**3 * x[2]**2 * x[3] \
             + x[0]    * x[1]**4 * x[2]**3 * x[3]**2 * x[4]
    return result

In [10]:
start_time = time.time()

for i, x in enumerate(param_values):
    y[i] = evaluate_model(x)

eval_time_numba = time.time() - start_time

print(f"Model evaluation took {eval_time_numba:.6f} seconds")

Model evaluation took 1.090427 seconds


In [11]:
print(f"Using numba resulted in acceleration by "
      f"{eval_time / eval_time_numba:.2f} times.")

Using numba resulted in acceleration by 3.68 times.
